# Data Inspection and Validation
***Loading data, vetting its quality, and understanding its shape***

## Overview
This intro notebook will use cuDF and hvplot (with bokeh charts) to load a public bike share dataset and get a general sense of what it contains, then run some cursory visualization to validate that the data is free of issues.

### cuDF and hvplot
- [cuDF](https://docs.rapids.ai/api/cudf/stable/), the core of RAPIDS, is a Python GPU DataFrame library (built on the Apache Arrow columnar memory format) for loading, joining, aggregating, filtering, and otherwise manipulating data in a pandas-like API.
- [hvplot](https://hvplot.holoviz.org/) is a high-level plotting API for the PyData ecosystem built on [HoloViews](http://holoviews.org/).

## Imports
Let's first make sure the necessary imports are present to load.

In [28]:
import cudf
import hvplot.cudf
import cupy
import pandas as pd

### Data Size and GPU Speedups
This tutorial's dataset size is about `2.1GB` unzipped and contains about `9 million rows`. While this will do for a tutorial, its still too small to get a sense of the speed up possible with GPU acceleration. We've created a larger `300 million row` [2010 Census Visualization](https://github.com/rapidsai/plotly-dash-rapids-census-demo) application available through the RAPIDS [GitHub page](https://github.com/rapidsai) as another demo. 

## Loading Data into cuDF
We need to download and extract the sample data we will use for this tutorial. This notebook uses the Kaggle [Chicago Divvy Bicycle Sharing Data](https://www.kaggle.com/yingwurenjian/chicago-divvy-bicycle-sharing-data) dataset. Once the `data.csv` file is downloaded and unzipped, point the paths below at the location:


In [4]:
from pathlib import Path

DATA_DIR = Path("../data")
FILENAME = Path("data.csv")

We now read the .csv file into the GPU cuDF Dataframe (which behaves similar to a Pandas dataframe). 

In [5]:
df = cudf.read_csv(DATA_DIR / FILENAME)

## Mapping out the Data Shape
CuDF supports all the standard Pandas operations for a quick look at the data e.g. to see the total number of rows...

In [4]:
len(df)

9495235

Or to inspect the column headers and first few rows...

In [5]:
df.head()

,trip_id,year,month,week,day,hour,usertype,gender,starttime,stoptime,...,from_station_id,from_station_name,latitude_start,longitude_start,dpcapacity_start,to_station_id,to_station_name,latitude_end,longitude_end,dpcapacity_end
0,2355134,2014,6,27,0,23,Subscriber,Male,2014-06-30 23:57:00,2014-07-01 00:07:00,...,131,Lincoln Ave & Belmont Ave,41.939365,-87.668385,15.0,303,Broadway & Cornelia Ave,41.945512,-87.645980,15.0
1,2355133,2014,6,27,0,23,Subscriber,Male,2014-06-30 23:56:00,2014-07-01 00:00:00,...,282,Halsted St & Maxwell St,41.864580,-87.646930,15.0,22,May St & Taylor St,41.869482,-87.655486,15.0
2,2355130,2014,6,27,0,23,Subscriber,Male,2014-06-30 23:33:00,2014-06-30 23:35:00,...,327,Sheffield Ave & Webster Ave,41.921687,-87.653714,19.0,225,Halsted St & Dickens Ave,41.919936,-87.648830,15.0
3,2355129,2014,6,27,0,23,Subscriber,Female,2014-06-30 23:26:00,2014-07-01 00:24:00,...,134,Peoria St & Jackson Blvd,41.877749,-87.649633,19.0,194,State St & Wacker Dr,41.887155,-87.627750,11.0
4,2355128,2014,6,27,0,23,Subscriber,Female,2014-06-30 23:16:00,2014-06-30 23:26:00,...,320,Loomis St & Lexington St,41.872187,-87.661501,15.0,134,Peoria St & Jackson Blvd,41.877749,-87.649633,19.0


Or to see the full list of columns...

In [6]:
df.columns

Index(['trip_id', 'year', 'month', 'week', 'day', 'hour', 'usertype', 'gender',
       'starttime', 'stoptime', 'tripduration', 'temperature', 'events',
       'from_station_id', 'from_station_name', 'latitude_start',
       'longitude_start', 'dpcapacity_start', 'to_station_id',
       'to_station_name', 'latitude_end', 'longitude_end', 'dpcapacity_end'],
      dtype='object')

Or see how many trips were made by subscribers.

In [7]:
df.groupby("usertype").size()

usertype
Customer         1277
Dependent         178
Subscriber    9493780
dtype: int32

## Improving Data Utility
Now that we have a basic idea of how big our dataset is and what it contains, we want to start making the data more meaningful. This task can vary from removing unnecessary columns, mapping values to be more human readable, or formatting them to be understood by our tools.  

Having looked at the `df.head()` above, the first thing we might want is to re-load the data, parsing the start-stop time columns as more usable datetimes types: 

In [6]:
df = cudf.read_csv(DATA_DIR / FILENAME, parse_dates=('starttime', 'stoptime'))

One thing we will want to do is to look at trips by day of week. Now that we have real datetime columns, we can use `dt.weekday` to add a `weekday` column to our `cudf` Dataframe:

In [7]:
df["weekday"] = df['starttime'].dt.weekday

## Inspecting Data Quality and Distribution
Another important step is getting a sense of the quality of the dataset. As these datasets are often larger than is feasible to look through row by row, mapping out the distribution of values early on helps find issuse that can derail an analysis later.

Some examples are gaps in data, unexpected or empty value types, infeasible values, or incorrect projections. 

## Gender and Subsriber Columns
We could do this in a numerical way, such as getting the totals from the 'gender' data column as a table:

In [8]:
mf_counts = df.groupby("gender").size().rename("count").reset_index()
mf_counts

,gender,count
0,Female,2378675
1,Male,7116560


While technically functional as a table, taking values and visualizating them as bars help to intuitively show the scale of the difference faster:

In [9]:
mf_counts.hvplot.bar("gender","count").opts(title="Total trips by gender")

:Bars   [gender]   (count)

### A Note on Preattentive Attributes
This subconcious ability to quickly recognize patterns is due to our brain's natural ability to find [preattentive attributes](http://daydreamingnumbers.com/blog/preattentive-attributes-example/), such as height, orientation, or color. Imagine 100 values in a table and 100 in a bar chart and how quickly you would be albe to find the smallest and largest values in either.

### Try It out
Now try using [hvplot's user guide](https://hvplot.holoviz.org/user_guide/Plotting.html) and our examples to create a hvplot that shows the distribution of `Subscriber` types:

In [12]:
# code here

The above data columns maybe show some potentially useful disparities, but without supplimental data, it would be hard to have a follow up questions.


## Trip Starts
Instead, another question we might want to ask is how many trip starts are there per day of the week? We can group the `cudf` Dataframe and call `hvplot.bar` directly the result:

In [13]:
day_counts = df.groupby("weekday").size().rename("count").reset_index()
day_counts.hvplot.bar("weekday", "count").opts(title="Trip starts, per Week Day", yformatter="%0.0f")

:Bars   [weekday]   (count)

With 0-4 being a weekday, and 5-6 being a weekend, there is a clear drop off of ridership on the weekends. Lets note that!


## Trips by Duration
Another quick look we can generate is to see the overall distribution of trip durations, this time using `hvplot.hist`:

In [10]:
# We selected an arbitrary 50 for bin size, try and see patterns with other sizes
df.hvplot.hist(y="tripduration", bins=50).opts(
    title="Trips Duration Histrogram", yformatter="%0.0f"
)

:Histogram   [tripduration]   (tripduration_count)

Clearly, most trips are less than 15 minuites long. 

`hvplot` also makes it simple to interrogate different dimensions. For example, we can add `groupby="month"` to our call to `hvplot.hist`, and automatically get a slider to see a histogram specific to each month:

In [15]:
df.hvplot.hist(y="tripduration", bins=50, groupby="month").opts(
    title="Trips Duration Histrogram by Month", yformatter="%0.0f", width=400
)

:DynamicMap   [month]
   :Histogram   [tripduration]   (tripduration_count)

By scrubbing between the months we can start to see a pattern of slightly longer trip durations emerge during the summer months.



## Trips vs Temperatures
Lets follow up on this by using `hvplot` to generate a KDE distributions using our `cudf` Dataframes for 9 million trips:

In [22]:
df.hvplot.kde(y="temperature").opts(title="Distribution of trip temperatures")


:Distribution   [temperature]   (Density)

Clearly most trips occur around a temperature sweet spot of around 65-80 degrees.


The `hvplot.heatmap` method can group in two dimensions and colormap according to aggregations on those groups. Here we see *average* trip duration by year and month: 

In [17]:
df.hvplot.heatmap(x='month', y='year', C='tripduration', 
                  reduce_function=cudf.DataFrame.mean , colorbar=True, cmap="Viridis")

:HeatMap   [month,year]   (tripduration)

So what we saw hinted at with the trip duration slider is much more clearly shown in this literal heatmap *(ba-dom-tss)*. 



## Trip Geography
Temperature and months aside, we might also want to bin the data geographically to check for anomalies. The `hvplot.hexbin` can show the counts for trip starts overlaid on a tile map:

In [18]:
df.hvplot.hexbin(x='longitude_start', y='latitude_start', geo=True, tiles="OSM").opts(width=600, height=600)

:Overlay
   .Tiles.I    :Tiles   [x,y]
   .HexTiles.I :HexTiles   [longitude_start,latitude_start]

Interestingly there seems to be a strong concentration of trips in a core area that radiate outwards. Lets take note of that. 

The location of the data compared to a current system map also seems to show that everything is where it should be, without any extraneous data points or off map projections:

<img src="../images/DivvyBikesStation_ map.png" />

## Data Cleanup
Based on our inspection, this dataset is uncommonly well formatted and of high quality. But a little cleanup and formatting aids will make some things simpler in future notebooks. 

One thing that is missing is a list of just station id's and their coordinates. Let's generate that and save it for later. First, let's group by all the unique "from" and "to" station id values:

In [19]:
from_ids = df.groupby("from_station_id")
to_ids = df.groupby("to_station_id")

It's possible (but unlikely) that a particular station is only a sink or source for trips. For good measure, let's make sure the group keys are identical:

In [20]:
set(from_ids.groups) == set(to_ids.groups)

/home/allan/anaconda3/envs/rapids16a/lib/python3.8/site-packages/cudf/core/groupby/groupby.py:85: UserWarning: GroupBy.groups() performance scales poorly with number of groups. Got 586 groups.
  warnings.warn(


True

Each group has items for a single station, which all have the same lat/lon. So let's make a new DataFrame by taking a representative from each group, then rename some columns:

In [21]:
stations = from_ids.nth(1).to_pandas()
stations.index.name = "station_id"
stations.rename(columns={"latitude_start": "lat", "longitude_start": "lon"}, inplace=True)
stations = stations.reset_index().filter(["station_id", "lat", "lon"])
stations

,station_id,lat,lon
0,2,41.872293,-87.624091
1,3,41.867226,-87.615355
2,4,41.856268,-87.613348
3,5,41.873958,-87.627739
4,6,41.885042,-87.612795
...,...,...,...
581,622,41.900363,-87.696704
582,623,41.872773,-87.623981
583,624,41.876268,-87.629155
584,625,42.041691,-87.680687


Finally write the results to "stations.csv" in our data directory:

In [1]:
stations.to_csv(DATA_DIR / "stations.csv", index=False)

NameError: name 'stations' is not defined

## Summary of the Data
Overall this is an interesting and useful dataset. Our preliminary vetting found no issues with quality and already started to hint at areas to investigate:

- Weekday vs Weekend trip counts
- Bike trips vs weather correlation 
- Core vs Outward trip concentrations 

We will follow up with these findings in our next notebook.